In [44]:
import random

# Define the database sizes
D1K_SIZE = 5
D10K_SIZE = 10000
D50K_SIZE = 50000
D100K_SIZE = 100000

In [12]:

# Define the database file names
D1K_FILE = "D1K.txt"
D10K_FILE = "D10K.txt"
D50K_FILE = "D50K.txt"
D100K_FILE = "D100K.txt"


In [13]:

# Define the function to generate a random transaction
def gen_transaction(num):
    transaction = []
    for i in range(num):
        # Generate a random item index between 0 and 99
        item_index = random.randint(0, 99)
        transaction.append(item_index)
    return transaction


In [45]:

# Define the function to generate a transactional database
def gen_database(file_name, db_size):
    with open(file_name, 'w') as db_file:
        for i in range(db_size):
            # Generate a random transaction size between 5 and 15
            transaction_size = random.randint(5, 15)
            # Generate a random transaction and write it to the file
            transaction = gen_transaction(transaction_size)
            db_file.write(" ".join([f"i{item}" for item in transaction]) + "\n")


In [46]:

# Generate the four transactional databases
gen_database(D1K_FILE, D1K_SIZE)
# gen_database(D10K_FILE, D10K_SIZE)
# gen_database(D50K_FILE, D50K_SIZE)
# gen_database(D100K_FILE, D100K_SIZE)



In [16]:

from collections import defaultdict
from itertools import combinations

# Define a function to read a transactional database from a file
def read_database(file_name):
    with open(file_name, 'r') as db_file:
        transactions = [set(line.strip().split()) for line in db_file]
    return transactions

In [65]:
# Define the Apriori algorithm function
def apriori(db, min_support):
    # Step 1: Find frequent 1-itemsets
    item_counts = defaultdict(int)
    for transaction in db:
        for item in transaction:
            item_counts[item] += 1
    num_transactions = len(db)
    freq_k_itemsets = {frozenset([item]): count / num_transactions for item, count in item_counts.items() if count / num_transactions >= min_support}

    frequent_itemsets = freq_k_itemsets
    # Step 2: Generate k-itemsets and find frequent ones until no more frequent itemsets are found
    k = 1
    while len(freq_k_itemsets) > 0:
        k += 1

        # Generate candidate k-itemsets from frequent (k-1)-itemsets
        candidate_itemsets = set([itemset1.union(itemset2) for itemset1 in frequent_itemsets for itemset2 in frequent_itemsets if len(itemset1.union(itemset2)) == k])
        # Count the support of each candidate itemset
        item_counts = defaultdict(int)
        for transaction in db:
            for candidate_itemset in candidate_itemsets:
                if candidate_itemset.issubset(transaction):
                    item_counts[candidate_itemset] += 1
        # Find frequent k-itemsets
        freq_k_itemsets = {itemset: count / num_transactions for itemset, count in item_counts.items() if count / num_transactions >= min_support}
        frequent_itemsets.update(freq_k_itemsets)

        return frequent_itemsets, k


# db = read_database('D1K.txt')
# frequent_itemsets = apriori(db, 0.01)

In [66]:
# Define a function to write frequent itemsets to a file
def write_frequent_itemsets(file_name, frequent_itemsets):
    with open(file_name, 'w') as freq_file:
        for itemset, support in frequent_itemsets.items():
            if len(itemset) == 1: continue
            freq_file.write("{" + ", ".join([f"{item}" for item in itemset]) + "} " + f"{support:.2%}\n")


In [67]:

# Define the main function to run the Apriori algorithm on a database and save frequent itemsets to a file
def main(db_file_name, min_support, apriori_fn=apriori):
    print("1", db_file_name)
    db = read_database(db_file_name)
    frequent_itemsets, k = apriori_fn(db, min_support)
    print(f'k = {k-1}')
    freq_file_name = db_file_name.replace(".txt", f"_AprioriAlgo_{int(min_support*100)}.freq")
    write_frequent_itemsets(freq_file_name, frequent_itemsets)
    print(f"Number of transactions found: {len(frequent_itemsets)}\n")


In [74]:
# Test the program using the first database with min_support = 1%
main("D1K.txt", 0.21)

1 D1K.txt
k = 1
Number of transactions found: 25



---

## Idea 1

In [72]:
# Define the Apriori algorithm function
def apriori_idea1(db, min_support):
    # Step 1: Find frequent 1-itemsets
    item_counts = defaultdict(int)
    for transaction in db:
        for item in transaction:
            item_counts[item] += 1
    num_transactions = len(db)
    freq_k_itemsets_all = {frozenset([item]): count / num_transactions for item, count in item_counts.items()}
    freq_k_itemsets = {frozenset([item]): count / num_transactions for item, count in item_counts.items() if count / num_transactions >= min_support}
    db_all_freq1 = []  # db with all transactions with freq itemset
    for transaction in db:
        satisfy_supp = False
        for it in transaction:
            if freq_k_itemsets_all[frozenset({it})] >= min_support:
                satisfy_supp = True
        if satisfy_supp:
            db_all_freq1.append(transaction)

    frequent_itemsets = freq_k_itemsets
    # Step 2: Generate k-itemsets and find frequent ones until no more frequent itemsets are found
    k = 1
    while len(freq_k_itemsets) > 0:
        k += 1

        # Generate candidate k-itemsets from frequent (k-1)-itemsets
        candidate_itemsets = set([itemset1.union(itemset2) for itemset1 in frequent_itemsets for itemset2 in frequent_itemsets if len(itemset1.union(itemset2)) == k])
        # Count the support of each candidate itemset
        item_counts = defaultdict(int)
        for transaction in db_all_freq1:
            for candidate_itemset in candidate_itemsets:
                if candidate_itemset.issubset(transaction):
                    item_counts[candidate_itemset] += 1
        # Find frequent k-itemsets
        freq_k_itemsets = {itemset: count / num_transactions for itemset, count in item_counts.items() if count / num_transactions >= min_support}
        frequent_itemsets.update(freq_k_itemsets)

        return frequent_itemsets, k

In [73]:
main('D1K.txt', .21, apriori_fn=apriori_idea1)

1 D1K.txt
k = 1
Number of transactions found: 25



---